In [51]:
library(lme4)
library(simr)
library(dplyr)
library(broom)
install.packages("sjPlot")
library(sjPlot)
install.packages("broom.mixed")
library(broom.mixed)
library(htmlTable)
library(dplyr)
library(stringr)
# library(gt)


The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpqTyJuM/downloaded_packages

The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpqTyJuM/downloaded_packages


In [52]:
library(car)
install.packages("stargazer")

library(stargazer)


The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//RtmpqTyJuM/downloaded_packages


In [56]:
file_path_total <- 'pilot_total_stats/correct_wm_ltm.csv'
file_path_5 <- 'pilot5_stats/correct_wm_ltm.csv'
file_path_6 <- 'pilot6_stats/correct_wm_ltm.csv'

df_total <- read.csv(file_path_total) 
df5 <- read.csv(file_path_5) 
df6 <- read.csv(file_path_6) 
# %>%
#     select(it_sim_dis_diff, it_sim_dis_diff_sq, reliability, v2_sim_dis_diff, v2_sim_dis_diff_sq, participant, validity, resp_correct, Accuracy, validity_binary, reliability_binary, v2_sim_dis_attend, v2_sim_dis_attend_z, v2_sim_dis_attend_sq_z, it_sim_dis_attend, it_sim_dis_attend_z, it_sim_dis_attend_sq_z, v2_sim_dis_unattend, v2_sim_dis_unattend_z, v2_sim_dis_unattend_sq_z, it_sim_dis_unattend, it_sim_dis_unattend_z, it_sim_dis_unattend_sq_z, it_sim_dis_diff_z, v2_sim_dis_diff_z,it_sim_dis_diff_sq_z, v2_sim_dis_diff_sq_z )

In [57]:
control <- lmerControl(optimizer = "bobyqa")#, optCtrl = list(maxfun = 10000))

In [62]:
df <- df_total

In [63]:
run_ltm_models <- function(df) {
  
    ltm_abs_split <- glmer(recall_answ ~ (it_ltm_sign_z + it_ltm_sign_sq_z) * it_pos_neg_ltm_abs * reliability_binary_z *  
                      attend_binary_z + (v2_ltm_sign_z + v2_ltm_sign_sq_z) * v2_pos_neg_ltm_abs * reliability_binary_z *  
                      attend_binary_z + tested_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
    ltm_rel_split <- glmer(recall_answ ~ (it_ltm_diff_sign_z + it_ltm_diff_sign_sq_z) * it_pos_neg_ltm * reliability_binary_z *  
                      attend_binary_z + (v2_ltm_diff_sign_z + v2_ltm_diff_sign_sq_z) * v2_pos_neg_ltm * reliability_binary_z *  
                      attend_binary_z + tested_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
    ltm_abs <- glmer(recall_answ ~ (it_ltm_z + it_ltm_sq_z)  * reliability_binary_z *  
                      attend_binary_z + (v2_ltm_z + v2_ltm_sq_z) * reliability_binary_z *  
                      attend_binary_z + tested_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
    ltm_rel <- glmer(recall_answ ~ (it_ltm_diff_z + it_ltm_diff_sq_z)  * reliability_binary_z *  
                      attend_binary_z + (v2_ltm_diff_z + v2_ltm_diff_sq_z) * reliability_binary_z *  
                      attend_binary_z + tested_binary_z + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))
     return(list(
    ltm_abs_split = ltm_abs_split,
    ltm_rel_split = ltm_rel_split,
    ltm_abs       = ltm_abs,
    ltm_rel       = ltm_rel
  ))
}

In [64]:
results_old_stims <- run_ltm_models(df6)
summary(results_old_stims$ltm_abs_split)




Correlation matrix not shown by default, as p = 45 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
recall_answ ~ (it_ltm_sign_z + it_ltm_sign_sq_z) * it_pos_neg_ltm_abs *  
    reliability_binary_z * attend_binary_z + (v2_ltm_sign_z +  
    v2_ltm_sign_sq_z) * v2_pos_neg_ltm_abs * reliability_binary_z *  
    attend_binary_z + tested_binary_z + (1 | participant)
   Data: df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 5e+05),  
    calc.derivs = FALSE)

     AIC      BIC   logLik deviance df.resid 
  7542.4   7861.1  -3725.2   7450.4     7504 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.8935  0.1733  0.3750  0.5585  1.7849 

Random effects:
 Groups      Name        Variance Std.Dev.
 participant (Intercept) 0.5652   0.7518  
Number of obs: 7550, groups:  participant, 132

Fixed effects:
                                                                         Estimate
(Intercept)                                

In [65]:
results_new_stims <- run_ltm_models(df5)
results_total <- run_ltm_models(df_total)


In [86]:
extract_significance <- function(model_summary) {
  if("coef.table" %in% names(model_summary)) {
    coef_table <- as.data.frame(model_summary$coef.table)
    p_values <- coef_table[, "Pr(>|z|)"] # or Pr(>|t|) for lmer
  } else if("coefficients" %in% names(model_summary)) {
    coef_table <- as.data.frame(model_summary$coefficients)
    p_values <- coef_table[, "Pr(>|z|)"] # or Pr(>|t|) for lm
  } else {
    stop("Unsupported model summary format")
  }
  
  var_names <- rownames(coef_table)
  
  stars <- sapply(p_values, function(p) {
    if(is.na(p)) return("")
    if(p < 0.001) return("***")
    if(p < 0.01) return("**")
    if(p < 0.05) return("*")
    if(p < 0.1) return(".")
    return("")
  })
  
  result <- data.frame(
    variable = var_names,
    significance = stars,
    p_value = p_values,
    stringsAsFactors = FALSE
  )
  
  return(result)
}

compare_models <- function(model_list, model_names, 
                          include_p_values = FALSE,
                          only_significant = TRUE,
                          significance_level = 0.1) {
  
  sig_list <- lapply(model_list, extract_significance)
  names(sig_list) <- model_names
  
  all_vars <- unique(unlist(lapply(sig_list, function(x) x$variable)))
  
  result_df <- data.frame(Variable = all_vars, stringsAsFactors = FALSE)
  
  for(model_name in model_names) {
    model_sig <- sig_list[[model_name]]
    if(include_p_values) {
      result_df[[model_name]] <- sapply(all_vars, function(var) {
        idx <- which(model_sig$variable == var)
        if(length(idx) == 0) return("")
        p_val <- model_sig$p_value[idx]
        stars <- model_sig$significance[idx]
        if(stars == "") return("")
        return(paste0(stars, " (p=", sprintf("%.4f", p_val), ")"))
      })
    } else {
      result_df[[model_name]] <- sapply(all_vars, function(var) {
        idx <- which(model_sig$variable == var)
        if(length(idx) == 0) return("")
        return(model_sig$significance[idx])
      })
    }
  }
  
  if(only_significant) {
    has_significance <- apply(result_df[, -1, drop = FALSE], 1, function(row) {
      any(str_detect(row, "\\*|\\+|\\."))
    })
    result_df <- result_df[has_significance, ]
  }
  
  html_table <- htmlTable(
    result_df,
    caption = "Significance Comparison Across Models",
    rnames = FALSE,
    css.cell = "padding: 8px;",
    css.table = "border-collapse: collapse; width: 100%; font-family: Arial, sans-serif;",
    css.header = "background-color: #f2f2f2; font-weight: bold; border: 1px solid #dddddd;",
    css.rgroup = "border-bottom: 1px solid #dddddd;",
    css.tspanner = "font-weight: bold; text-align: center; border-bottom: 2px solid #dddddd;",
    css.class = "table-hover"
  )
  
  return(html_table)
}


In [87]:
ltm_abs_split_old <- summary(results_old_stims$ltm_abs_split)
ltm_abs_split_new <- summary(results_new_stims$ltm_abs_split)
ltm_abs_split_total <- summary(results_total$ltm_abs_split)

comparison_table_abs_split <- compare_models(
  model_list = list(ltm_abs_split_old, ltm_abs_split_new, ltm_abs_split_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
writeLines(comparison_table_abs_split, "comparison_table_abs_split.html")

In [75]:
ltm_rel_split_old <- summary(results_old_stims$ltm_rel_split)
ltm_rel_split_new <- summary(results_new_stims$ltm_rel_split)
ltm_rel_split_total <- summary(results_total$ltm_rel_split)

comparison_table_rel_split <- compare_models(
  model_list = list(ltm_rel_split_old, ltm_rel_split_new, ltm_rel_split_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
writeLines(comparison_table_rel_split, "comparison_table_rel_split.html")

In [76]:
ltm_abs_old <- summary(results_old_stims$ltm_abs)
ltm_abs_new <- summary(results_new_stims$ltm_abs)
ltm_abs_total <- summary(results_total$ltm_abs)

comparison_table_abs <- compare_models(
  model_list = list(ltm_abs_old, ltm_abs_new, ltm_abs_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
writeLines(comparison_table_abs, "comparison_table_abs.html")

In [77]:
ltm_rel_old <- summary(results_old_stims$ltm_rel)
ltm_rel_new <- summary(results_new_stims$ltm_rel)
ltm_rel_total <- summary(results_total$ltm_rel)

comparison_table_rel <- compare_models(
  model_list = list(ltm_rel_old, ltm_rel_new, ltm_rel_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,  # Change to FALSE for just stars
  only_significant = TRUE
)

# Save to HTML file for inclusion in presentations
writeLines(comparison_table_rel, "comparison_table_rel.html")

In [81]:
IRdisplay::display_html(comparison_table_abs_split)
IRdisplay::display_html(comparison_table_abs)


Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0001) 
 *** (p=0.0000) 
 
 
 it_ltm_sign_z 
 
 ** (p=0.0054) 
 ** (p=0.0099) 
 
 
 it_pos_neg_ltm_abs 
 
 * (p=0.0234) 
 * (p=0.0166) 
 
 
 tested_binary_z 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_ltm_sign_sq_z:it_pos_neg_ltm_abs 
 
 *** (p=0.0004) 
 *** (p=0.0008) 
 
 
 it_ltm_sign_sq_z:reliability_binary_z 
 
 
 . (p=0.0511) 
 
 
 it_pos_neg_ltm_abs:reliability_binary_z 
 * (p=0.0365) 
 
 
 
 
 it_ltm_sign_z:it_pos_neg_ltm_abs:reliability_binary_z 
 . (p=0.0960) 
 
 . (p=0.0546) 
 
 
 reliability_binary_z:attend_binary_z:v2_ltm_sign_z:v2_pos_neg_ltm_abs 
 
 
 . (p=0.0708)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_ltm_z 
 
 . (p=0.0794) 
 
 
 
 it_ltm_sq_z 
 
 
 * (p=0.0376) 
 
 
 attend_binary_z 
 
 
 . (p=0.0772) 
 
 
 tested_binary_z 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_ltm_z:reliability_binary_z 
 ** (p=0.0071) 
 
 * (p=0.0171) 
 
 
 it_ltm_sq_z:reliability_binary_z 
 . (p=0.0638) 
 
 . (p=0.0610) 
 
 
 reliability_binary_z:v2_ltm_z 
 
 * (p=0.0399) 
 * (p=0.0289) 
 
 
 reliability_binary_z:v2_ltm_sq_z 
 
 * (p=0.0122) 
 ** (p=0.0080) 
 
 
 it_ltm_z:reliability_binary_z:attend_binary_z 
 *** (p=0.0007) 
 
 * (p=0.0194) 
 
 
 it_ltm_sq_z:reliability_binary_z:attend_binary_z 
 ** (p=0.0030) 
 
 * (p=0.0474)

In [82]:
IRdisplay::display_html(comparison_table_rel_split)
IRdisplay::display_html(comparison_table_rel)


Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_ltm_diff_sign_z 
 ** (p=0.0011) 
 
 ** (p=0.0020) 
 
 
 it_ltm_diff_sign_sq_z 
 
 * (p=0.0409) 
 
 
 
 attend_binary_z 
 
 
 . (p=0.0760) 
 
 
 v2_ltm_diff_sign_sq_z 
 ** (p=0.0089) 
 
 
 
 
 tested_binary_z 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_ltm_diff_sign_z:it_pos_neg_ltm 
 
 . (p=0.0796) 
 
 
 
 it_ltm_diff_sign_sq_z:reliability_binary_z 
 . (p=0.0741) 
 
 . (p=0.0628) 
 
 
 it_pos_neg_ltm:reliability_binary_z 
 . (p=0.0886) 
 
 . (p=0.0604) 
 
 
 it_ltm_diff_sign_z:attend_binary_z 
 
 . (p=0.0658) 
 
 
 
 reliability_binary_z:v2_ltm_diff_sign_z 
 
 ** (p=0.0098) 
 * (p=0.0487) 
 
 
 reliability_binary_z:v2_ltm_diff_sign_sq_z 
 
 . (p=0.0872) 
 * (p=0.0281) 
 
 
 attend_binary_z:v2_pos_neg_ltm 
 
 
 . (p=0.1000) 
 
 
 it_ltm_diff_sign_z:reliability_binary_z:attend_binary_z 
 
 . (p=0.0963) 
 * (p=0.0261) 
 
 
 it_ltm_diff_sign_sq_z:reliability_binary_z:attend_binary_z 
 * (p=0.0153) 
 
 . (p=0.0948) 
 
 
 reliability_binary_z:v2_ltm_diff_sign_z:v2_pos_neg_ltm 
 
 . (p=0.0739) 
 * (p=0.0382) 
 
 
 reliability_binary_z:v2_ltm_diff_sign_sq_z:v2_pos_neg_ltm 
 
 . (p=0.0926) 
 
 
 
 reliability_binary_z:attend_binary_z:v2_ltm_diff_sign_z 
 * (p=0.0150) 
 
 
 
 
 reliability_binary_z:attend_binary_z:v2_ltm_diff_sign_z:v2_pos_neg_ltm 
 * (p=0.0476)

Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_ltm_diff_sq_z 
 ** (p=0.0016) 
 
 ** (p=0.0013) 
 
 
 attend_binary_z 
 
 
 . (p=0.0817) 
 
 
 tested_binary_z 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 it_ltm_diff_sq_z:attend_binary_z 
 
 * (p=0.0474) 
 
 
 
 reliability_binary_z:v2_ltm_diff_sq_z 
 
 ** (p=0.0027) 
 * (p=0.0176) 
 
 
 attend_binary_z:v2_ltm_diff_sq_z 
 
 . (p=0.0704) 
 . (p=0.0734) 
 
 
 it_ltm_diff_sq_z:reliability_binary_z:attend_binary_z 
 
 
 * (p=0.0245) 
 
 
 reliability_binary_z:attend_binary_z:v2_ltm_diff_sq_z 
 . (p=0.0561)